# cuDF

现在让我们转向一些更高级的 API，从 [cuDF](https://github.com/rapidsai/cudf) 开始。与 `pandas` 类似，`cudf` 库是一个用于处理表格数据集的数据框架包。

数据被加载到 GPU 上，所有操作都使用 GPU 计算完成，但 `cudf` 的 API 对 `pandas` 用户来说应该非常熟悉。

In [ ]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

In [ ]:
import cudf

## 数据加载

在本教程中，我们在 `data/` 中存储了一些数据。这些数据大多太小，无法真正从 GPU 加速中受益，但让我们还是来探索一下。

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/NVIDIA/accelerated-computing-hub/refs/heads/main/gpu-python-tutorial/data/pageviews_small.csv", sep=" ")
df.head()

In [ ]:
pageviews = cudf.read_csv("https://raw.githubusercontent.com/NVIDIA/accelerated-computing-hub/refs/heads/main/gpu-python-tutorial/data/pageviews_small.csv", sep=" ")
pageviews.head()

这个 `pageviews.csv` 文件包含了来自维基百科不同语言版本的超过 `1M` 条页面浏览记录。

让我们重命名列并删除未使用的 `x` 列。

In [ ]:
pageviews.columns = ['project', 'page', 'requests', 'x']

pageviews = pageviews.drop('x', axis=1)

pageviews

接下来，让我们统计这个数据集中有多少条英语记录。

In [ ]:
print(pageviews[pageviews.project == 'en'].count())

然后让我们执行一个分组操作，按语言统计所有页面。

In [ ]:
grouped_pageviews = pageviews.groupby('project').count().reset_index()
grouped_pageviews

最后，让我们看看英语、法语、中文和波兰语的结果。

In [ ]:
print(grouped_pageviews[grouped_pageviews.project.isin(['en', 'fr', 'zh', 'pl'])])

如果您以前使用过 `pandas`，那么所有这些语法对您来说应该都很熟悉。就像 `cupy` 实现了大部分 `numpy` API 一样，`cudf` 实现了大部分 `pandas` API。

唯一的区别是我们所有的过滤和分组操作都在 GPU 而不是 CPU 上进行，从而获得更好的性能。

### 字符串操作

GPU 历来以数值计算著称，而不是用于处理更复杂的对象。在 cuDF 中，字符串操作也通过专门的内核得到加速。

这意味着像字符串大写这样的操作可以在 GPU 上并行化。

In [ ]:
pageviews[pageviews.project == 'en'].page.str.upper()

In [ ]:
pageviews_en = pageviews[pageviews.project == 'en']
print(pageviews_en.page.str.upper().head())

### 用户自定义函数（UDFs）

cuDF 还支持用户自定义函数（UDFs），这些函数可以在 GPU 上并行地映射到 Series 或 DataFrame 上。

UDFs 可以定义为接受单个值的纯 Python 函数。当我们调用 `.apply()` 时，这些函数将被 Numba 在运行时编译成可以在 GPU 上运行的代码。

In [ ]:
def udf(x):
    if x < 5:
        return 0
    return x

In [ ]:
pageviews.requests = pageviews.requests.apply(udf)

In [ ]:
pageviews.requests

也可以直接使用 Numba 编写接收输入列和输出列指针以及额外参数的内核。然后内核可以像我们在第2/3章中那样使用 `cuda.grid` 来获取要操作的索引。

然后我们使用 `.forall()` 将我们的内核映射到一个列上。

In [ ]:
pageviews['mul_requests'] = 0.0

In [ ]:
from numba import cuda

In [ ]:
@cuda.jit
def multiply(in_col, out_col, multiplier):
    i = cuda.grid(1)
    if i < in_col.size: # 边界检查
        out_col[i] = in_col[i] * multiplier

In [ ]:
multiply.forall(len(pageviews))(pageviews['requests'], pageviews['mul_requests'], 10.0)

In [ ]:
print(pageviews.head())

## 滑动窗口

cuDF 还支持在滑动窗口上应用内核。这实际上是一个一维模板，可以让我们基于邻居执行操作。

![](images/rolling-windows.png)

In [ ]:
def neigborhood_mean(window):
    c = 0
    for val in window:
        c += val
    return c / len(window)

In [ ]:
pageviews.requests.rolling(3, 1, True).apply(neigborhood_mean)